In [ ]:
!pip install google-api-python-client
!pip install oauth2client

In [9]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

API_KEY = "KEY"
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=API_KEY)

In [53]:
search_channel = youtube.search().list(
    q = "유랑쓰 YOURANGSS",
    part = "snippet",
    order = "relevance",
    maxResults = 50,
).execute()

In [52]:
def set_video(channelId, maxCount):
    search_response = youtube.search().list(
        order = 'date',
        part = 'snippet',
        channelId = channelId,
        maxResults = maxCount,
    ).execute()

    channel_response = youtube.channels().list(
        part = 'statistics',
        id = channelId,
    ).execute()

    video_ids = []
    for i in range(0, len(search_response['items'])):
        video_ids.append((search_response['items'][i]['id']['videoId']))

    channel_video_id = []
    channel_video_title = []
    channel_video_description = []
    channel_rating_view = []
    channel_rating_comment = []
    channel_rating_like = []
    channel_subscriber_count = []
    channel_thumbnail_url = []
    channel_published_date = []
    data_dicts = {}

    for k in range(0, len(search_response['items'])):
        video_ids_lists = youtube.videos().list(
            part = 'snippet, statistics',
            id = video_ids[k],
        ).execute()

        str_video_id = video_ids_lists['items'][0]['id']
        str_video_title = video_ids_lists['items'][0]['snippet'].get('title')
        str_video_description = video_ids_lists['items'][0]['snippet'].get('description')
        str_video_view = video_ids_lists['items'][0]['statistics'].get('viewCount')
        str_video_comment = video_ids_lists['items'][0]['statistics']['commentCount']
        str_video_like = video_ids_lists['items'][0]['statistics'].get('likeCount')
        str_video_subscriber = channel_response['items'][0]['statistics']['subscriberCount']
        str_video_thumbnail_url = str(video_ids_lists['items'][0]['snippet']['thumbnails']['high'].get('url'))
        str_published_date = str(video_ids_lists['items'][0]['snippet'].get('publishedAt'))

        channel_video_id.append(str_video_id)
        channel_video_title.append(str_video_title)
        channel_video_description.append(str_video_description)
        channel_rating_view.append(str_video_view)
        channel_rating_comment.append(str_video_comment)
        channel_rating_like.append(str_video_like)
        channel_subscriber_count.append(str_video_subscriber)
        channel_thumbnail_url.append(str_video_thumbnail_url)
        channel_published_date.append(str_published_date)

    data_dicts['id'] = channel_video_id
    data_dicts['title'] = channel_video_title
    data_dicts['description'] = channel_video_description
    data_dicts['view'] = channel_rating_view
    data_dicts['comment'] = channel_rating_comment
    data_dicts['like'] = channel_rating_like
    data_dicts['subscriber'] = channel_subscriber_count
    data_dicts['thumbnail'] = channel_thumbnail_url
    data_dicts['published_date'] = channel_published_date

    return data_dicts


In [59]:
youtuber = set_video('UCnVEtY_GfM7meN1-pyl7E7A', '25')

In [65]:
import pandas as pd

df = pd.DataFrame([youtuber['id'],
                  youtuber['title'],
                  youtuber['description'],
                  youtuber['view'],
                  youtuber['like'],
                  youtuber['comment'],
                  youtuber['subscriber'],
                  youtuber['thumbnail'],
                  youtuber['published_date']]).T
df.columns = ['id', 'title', 'description', 'view', 'like', 'comment', 'subscriber', 'thumbnail', 'published_date']
df

,id,title,description,view,like,comment,subscriber,thumbnail,published_date
0,SO8NLzTlwu8,"대한민국 부동산, 살기 좋은 동네를 찾아서",30대 부부의 부동산 고민\n#부동산 #부동산매매 #전세 #월세 #임장\n\n📸 인...,102377,2717,623,252000,https://i.ytimg.com/vi/SO8NLzTlwu8/hqdefault.jpg,2025-02-23T09:01:41Z
1,d-0efd13dq8,10년차 부부… 우린 왜 집이 없어?,30대 부부의 부동산 고민\n#부동산 #전세 #월세 #매매 \n\n* 본 영상은 비...,147012,3371,1288,252000,https://i.ytimg.com/vi/d-0efd13dq8/hqdefault.jpg,2025-02-15T01:06:45Z
2,ahvRNbRfUwA,여러분들은 대전에 대해 오해가 많으십니다...,대전 맛집 먹방 여행 브이로그\n#대전 #대전여행 #대전맛집 #대전먹방 #대전술집 ...,140807,3362,585,252000,https://i.ytimg.com/vi/ahvRNbRfUwA/hqdefault.jpg,2025-02-08T01:03:06Z
3,L6NhmVmQs5s,고물가시대 혜자로운 코스트코 장보기,동탄러의 코스트코 쇼핑 브이로그\n#코스트코 #일상 #브이로그\n\n📸 인스타그램 ...,166424,4248,912,252000,https://i.ytimg.com/vi/L6NhmVmQs5s/hqdefault.jpg,2025-02-01T01:16:22Z
4,BJDB2VAErWQ,발리 5성 호텔에 머물다 급하게 귀국한 이유,발리 우붓 여행 브이로그 5화\n#발리 #우붓 #마야우붓\n\n📍 영상 속 장소\n...,213231,3844,530,252000,https://i.ytimg.com/vi/BJDB2VAErWQ/hqdefault.jpg,2025-01-23T01:55:55Z
5,uUfWQU0p81g,역대급 하루를 보내고 발리밸리인지 정체 모를 병으로 앓아 눕다...,발리 우붓 여행 브이로그 4화\n#발리 #우붓 #발리투어\n\n(본 영상은 몰바니의...,129839,3216,548,252000,https://i.ytimg.com/vi/uUfWQU0p81g/hqdefault.jpg,2025-01-18T01:04:28Z
6,DN8lX_a2oNQ,"물가 급등한 발리, 집 구하고 살아본 솔직 후기",발리 우붓 여행 브이로그 3화\n#발리 #우붓 #한달살기\n\n📍 영상 속 장소\n...,156914,3772,565,252000,https://i.ytimg.com/vi/DN8lX_a2oNQ/hqdefault.jpg,2025-01-11T02:37:15Z
7,m1oy3ambTOg,지푸라기 잡는 심정으로 유명한 철학관에 간 사연,발리 우붓 여행 브이로그 2화\n#발리 #우붓 #한달살기\n\n(본 영상은 '스픽'...,219699,5347,1037,252000,https://i.ytimg.com/vi/m1oy3ambTOg/hqdefault.jpg,2025-01-07T01:04:12Z
8,-M5-PyWmh1A,전세계 살기 좋은 도시를 찾아서 🌎,발리 우붓 여행 브이로그 1화\n#자유 #발리 #우붓 #한달살기\n\n📍영상 속 장...,183386,4026,600,252000,https://i.ytimg.com/vi/-M5-PyWmh1A/hqdefault.jpg,2024-12-28T01:21:59Z
9,sGARDlf8NDM,베스트셀러가 되면 돈을 얼마나 벌까?,30대 부부 브이로그\n#베스트셀러 #수익 #브이로그 \n\n📚 도서 (유랑하는 자...,170742,5514,935,252000,https://i.ytimg.com/vi/sGARDlf8NDM/hqdefault.jpg,2024-11-29T06:00:25Z
